# Modelo de inteligencia artificial de PawSense

## Fase 1: Preparación del entorno

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import glob
import os
from PIL import Image

## Fase 2: Ingeniería de datos.

Aplicamos recorte de las imágenes para que el modelo aprenda exclusivamente de los perros no de otros posibles objetos de la foto (coches, personas, árboles, etc)

In [2]:
# Para modelo EfficientNetB0 espera img 224 x 224 pixeles y 3 canales de color
input_shape = (224, 224, 3)

# Establecemos numero de imagenes que lee a la vez (no muy alto)
batch_size = 32

# Patron buscar subcarpetas
xml_pattern = r"dataset_perros/annotations/Annotation/**/*"

# Para cada path parsea arquitectura xml y de ahi obtenemos desde la raiz las medidas del perro en las imagenes
for xmlpath in glob.glob(xml_pattern, recursive=True):
    # Para que no se salga de la carpeta
    if os.path.isdir(xmlpath):
        continue

    try:
        # Estructura, raiz y objeto
        tree = ET.parse(xmlpath)
        root = tree.getroot()

        obj = root.find('object')
        
        if obj is not None:

            # Sacamos medidas del perro en la foto
            box = obj.find('bndbox')
            raza = obj.find('name').text

            xmin = int(box.find('xmin').text)
            xmax = int(box.find('xmax').text)
            ymin = int(box.find('ymin').text)
            ymax = int(box.find('ymax').text)

            # Obtenemos los nombres de carpetas e imagenes directamente del nombre del archivo XML
            # Esto corrige el error donde las etiquetas <filename> o <folder> en el XML son incorrectas
            xml_filename = os.path.basename(xmlpath)
            imgname = os.path.splitext(xml_filename)[0] # Nombre archivo sin extension
            
            # El nombre de la carpeta padre del xml debe coincidir con la carpeta de la imagen
            imgfoldername = os.path.basename(os.path.dirname(xmlpath))

            # Patron para buscar imágenes por carpeta e imagen obtenida del nombre del archivo
            img_pattern = rf"dataset_perros/images/Images/{imgfoldername}/{imgname}.jpg"

            encontrados = glob.glob(img_pattern)
            if not encontrados:
                print(f"Procesando archivo: {xmlpath}")
                print(f"❌ No se encontró imagen para: {imgname} en {imgfoldername}")

            # Para cada path abre la imagen, asegura de que este en colores, redimensiona a lo especificado en cml (por si acaso), reajusta calidad y recorta la imágen.
            for imgpath in glob.glob(img_pattern, recursive=True):
                img = Image.open(imgpath).convert('RGB')

                # Reajustar medidas de imagen para evitar problemas de recorte
                width, height = img.size
                xml_w = int(root.find('size/width').text)
                xml_h = int(root.find('size/height').text)
                if width != xml_w or height != xml_h:
                    img = img.resize((xml_w, xml_h), Image.LANCZOS)
                
                # Recorte de imágen
                img_final = img.crop((xmin, ymin, xmax, ymax)).resize((224, 224), Image.LANCZOS)

                # Crea directorio con carpetas de nombres de raza
                
                output_dir = os.path.join("dataset_recortado", raza)
                os.makedirs(output_dir, exist_ok=True)
               
                # Guarda imágenes
                save_path = os.path.join(output_dir, f"{imgname}.jpg")
                img_final.save(save_path)
    except Exception as e:
        print(f"Saltando {xmlpath} por error: {e}")

## Fase 3: EDA - Visualización de datos.

- Gráfico de barras del top 10 y bottom 10 de razas para ver si hay desequilibrio.

- Mostrar fotos y clases para la clasificación

- Gráfico de "Relación de Aspecto" (Ancho vs Alto). Los Galgos son altos/finos, los Bulldogs son anchos. Esto justifica usar EfficientNet.